# PTT [科技工作版] 文章爬蟲

[PTT 科技工作版](https://www.ptt.cc/bbs/Tech_Job/index.html)

## 發送請求、解析回應

In [ ]:
import requests # 安裝“發送請求”套件
from bs4 import BeautifulSoup # 安裝“解析HTML”套件

In [ ]:
# 目標頁面
url = 'https://www.ptt.cc/bbs/Tech_Job/index.html'

In [ ]:
# 發送 GET 請求，指定 url 與 headers，將回應結果存在變數
response = requests.get(url)

In [ ]:
# 查看回應狀態代碼，200 代表成功
response.status_code

In [ ]:
# 查看回應內容
print(response.text)

In [ ]:
# 將回應的內容文字放在 BeautifulSoup 進行解析，存在 soup 變數中
soup = BeautifulSoup(response.text, "html.parser")
soup

In [ ]:
print(soup.prettify())  # 輸出排版美化後的 HTML 內容

## 擷取單一文章

### 找尋特定元件 find()
- 回傳最近的一個子元件（往下找，平行或上層的不包含）
- 可指定符合某屬性及屬性值  find(元件, {屬性: 屬性值})
- 若找不到則回傳 None
- select_one() 是另一種方法，跟 find() 效果相同，使用語法不同

In [ ]:
soup.find("div",{"class": "r-ent"})

In [ ]:
# 可連續使用多個方法（一層一層往下找）
soup.find("div",{"class": "r-ent"}).find("div",{"class": "title"})

### 取得內容文字 getText()

In [ ]:
#標題
soup.find("div",{"class": "r-ent"}).find("div",{"class": "title"}).getText()

In [ ]:
# 去除標題前後的換行符號
soup.find("div",{"class": "r-ent"}).find("div",{"class": "title"}).getText().strip('\n')

In [ ]:
#超連結
soup.find("div",{"class": "r-ent"}).find("div",{"class": "title"}).find("a")

### 取得指定屬性的屬性值 get()

In [ ]:
#超連結
soup.find("div",{"class": "r-ent"}).find("div",{"class": "title"}).find("a").get("href")

In [ ]:
#發文者
soup.find("div",{"class": "r-ent"}).find("div",{"class": "author"}).getText()

In [ ]:
#發文日期
soup.find("div",{"class": "r-ent"}).find("div",{"class": "date"}).getText()

In [ ]:
#推文數
soup.find("div",{"class": "r-ent"}).find("div",{"class": "nrec"}).getText()

In [ ]:
# 將重複的部分宣告為變數（免得每次都要打一長串）
article = soup.find("div",{"class": "r-ent"})

# 打印出這篇文章的標題、連結、發文者、日期、推文數
print(article.find("div",{"class": "title"}).getText())
print(article.find("div",{"class": "title"}).find("a").get("href"))
print(article.find("div",{"class": "author"}).getText())
print(article.find("div",{"class": "date"}).getText())
print(article.find("div",{"class": "nrec"}).getText())

## 擷取文章列表

### 找尋全部的特定元件 find_all()
- 回傳所有符合的子元件列表（資料型態為 List）
- 可指定符合某屬性及屬性值  find(元件, {屬性: 屬性值})
- 若找不到則回傳 None
- select() 是另一種方法，跟 find_all() 效果相同，使用語法不同

In [ ]:
soup.find_all("div",{"class": "r-ent"})

In [ ]:
# 查看其中一筆資料(一篇文章)
soup.find_all("div",{"class": "r-ent"})[0]

In [ ]:
# 檢查元件列表長度（代表找到的數量）
len(soup.find_all("div",{"class": "r-ent"}))

In [ ]:
# 存在一個變數
all_article = soup.find_all("div",{"class": "r-ent"})

### 使用迴圈走訪所有元件

In [ ]:
# 使用 for 迴圈走訪文章列表（List）
for article in all_article:
    if article.find("div",{"class": "title"}).find("a") is not None:
        # 打印出每一篇文章的標題、連結、發文者、日期、推文數
        print(article.find("div",{"class": "title"}).getText()) # 去除標題前後的換行符號
        print(article.find("div",{"class": "title"}).find("a").get("href"))
        print(article.find("div",{"class": "author"}).getText())
        print(article.find("div",{"class": "date"}).getText())
        print(article.find("div",{"class": "nrec"}).getText())

In [ ]:
# 將取得到的資料存在變數裡
for article in all_article:
    if article.find("div",{"class": "title"}).find("a") is not None:
        # 將每篇文章的標題、連結、發文者、日期、推文數存入變數
        title = article.find("div",{"class": "title"}).getText().strip('\n')
        link = article.find("div",{"class": "title"}).find("a").get("href") # 組合成完整的連結
        author = article.find("div",{"class": "author"}).getText()
        date = article.find("div",{"class": "date"}).getText()
        rec_num = article.find("div",{"class": "nrec"}).getText()

        print(title, link, author, date, rec_num)

## 資料整理＆輸出

### 將資料存成字典列表(List of Dictionary)
方便後續輸出成資料表(Excel Table)

In [ ]:
# 宣告一個空的文章列表(List)，用來存放文章資料（Dictionary）
article_list = []
for article in all_article:
    if article.find("div",{"class": "title"}).find("a") is not None:
        title = article.find("div",{"class": "title"}).getText().strip('\n')
        link = article.find("div",{"class": "title"}).find("a").get("href")
        link = "https://www.ptt.cc/bbs/Tech_Job" + article.find("div",{"class": "title"}).find("a").get("href")
        author = article.find("div",{"class": "author"}).getText()
        date = article.find("div",{"class": "date"}).getText()
        rec_num = article.find("div",{"class": "nrec"}).getText()

        # 宣告字典，欄位名稱(key)自訂，欄位值(value)等於變數
        article_dict = {
            "看板": "Tech_Job",    #預設固定欄位值
            "標題": title,
            "連結": link,
            "作者": author,
            "日期": date,
            "推文數": rec_num
        }
        # 打印出字典
        print(article_dict)
        # 將字典新增到列表當中
        article_list.append(article_dict)

In [ ]:
# 查看文章列表(List of Dictionary)
article_list

### 將資料存成實體檔案

In [ ]:
import pandas as pd # 安裝“資料處理”套件，使用縮寫

# 將字典列表存為 DataFrame
article_df = pd.DataFrame(article_list)
# 輸出為 csv 格式檔案
article_df.to_csv("PTT_Tech_Job.csv")

## 進階：模組化
寫成自訂函式 Function

In [ ]:
# 定義一個 Function
def ptt_crawler():

    # 目標頁面
    url = 'https://www.ptt.cc/bbs/Tech_Job/index.html'

    # 發送 GET 請求，指定 url，將回應結果存在變數
    response = requests.get(url)

    # 將回應的內容文字放在 BeautifulSoup 進行解析，存在 soup 變數中
    soup = BeautifulSoup(response.text, "html.parser")

    # 存在一個變數
    all_article = soup.find_all("div",{"class": "r-ent"})

    # 宣告一個空的文章列表(List)，用來存放文章資料（Dictionary）
    article_list = []
    for article in all_article:
        if article.find("div",{"class": "title"}).find("a") is not None:
            title = article.find("div",{"class": "title"}).getText().strip('\n')
            link = article.find("div",{"class": "title"}).find("a").get("href")
            link = "https://www.ptt.cc/bbs/Tech_Job" + article.find("div",{"class": "title"}).find("a").get("href")
            author = article.find("div",{"class": "author"}).getText()
            date = article.find("div",{"class": "date"}).getText()
            rec_num = article.find("div",{"class": "nrec"}).getText()

            # 宣告字典，欄位名稱(key)自訂，欄位值(value)等於變數
            article_dict = {
                "看板": "Tech_Job",    #預設固定欄位值
                "標題": title,
                "連結": link,
                "作者": author,
                "日期": date,
                "推文數": rec_num
            }
            # 將字典新增到列表當中
            article_list.append(article_dict)

    #回傳字典列表(List of Dictionary)
    return article_list

In [ ]:
ptt_crawler()

In [ ]:
data = ptt_crawler()

## 如果想要爬其他看版呢？ e.g. 股票、體育、表特、工作、八卦

In [ ]:
# 定義一個 Function，一個傳入參數：看板名稱
def ptt_crawler(board):

    # 目標頁面，將網址內的看板名稱改為變數（使用 f-strings）
    url = f'https://www.ptt.cc/bbs/{board}/index.html'

    # 設定標頭(頭部)資訊
    headers = {
        "cookie": "over18=1"    # 在 cookie 中存放資訊：已滿18歲
        # "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"
    }

    # 發送 GET 請求，指定 url 與 headers，將回應結果存在變數
    response = requests.get(url, headers = headers)

    # 將回應的內容文字放在 BeautifulSoup 進行解析，存在 soup 變數中
    soup = BeautifulSoup(response.text, "html.parser")

    # 存在一個變數
    all_article = soup.find_all("div",{"class": "r-ent"})

    # 宣告一個空的文章列表(List)，用來存放文章資料（Dictionary）
    article_list = []
    for article in all_article:
        if article.find("div",{"class": "title"}).find("a") is not None:
            title = article.find("div",{"class": "title"}).getText().strip('\n')
            # 連結也要改
            link = f"https://www.ptt.cc/bbs/{board}" + article.find("div",{"class": "title"}).find("a").get("href")
            author = article.find("div",{"class": "author"}).getText()
            date = article.find("div",{"class": "date"}).getText()
            rec_num = article.find("div",{"class": "nrec"}).getText()

            # 宣告字典，欄位名稱(key)自訂，欄位值(value)等於變數
            article_dict = {
                "看板": board,    # 將固定值改為變數
                "標題": title,
                "連結": link,
                "作者": author,
                "日期": date,
                "推文數": rec_num
            }
            # 將字典新增到列表當中
            article_list.append(article_dict)

    #回傳字典列表(List of Dictionary)
    return article_list

In [ ]:
ptt_crawler('Stock')